# ML Pipeline Preparation
Follow the instructions below to help you create your ML pipeline.
### 1. Import libraries and load data from database.
- Import Python libraries
- Load dataset from database with [`read_sql_table`](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.read_sql_table.html)
- Define feature and target variables X and Y

In [38]:
# import libraries
import pandas as pd
import numpy as np
import os
from sqlalchemy import create_engine

# download necessary NLTK data
import nltk
nltk.download(['punkt', 'wordnet', 'stopwords'])
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords 
import re

#sklearn
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import train_test_split as tts
from sklearn.ensemble import RandomForestClassifier
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from sklearn.pipeline import Pipeline
from sklearn.metrics import classification_report

[nltk_data] Downloading package punkt to /Users/jeffsan/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /Users/jeffsan/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/jeffsan/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [6]:
# load data from database
engine = create_engine('sqlite:///DisasterResponse.db')
df = pd.read_sql("SELECT * FROM messages", engine)
X = df[['message', 'genre']]
Y = df.drop(columns=['id', 'message', 'original','genre'])


In [51]:
#encoding genre by onehot
X_onehot = pd.get_dummies(X,columns=['genre'])

### 2. Write a tokenization function to process your text data

In [27]:
def tokenize(text):
    #remove punctuation
    text = re.sub(r"[^a-zA-Z0-9]", " ", text)
    
    #tokenize text
    tokens = word_tokenize(text)
    
    # initiate lemmatizer
    lemmatizer = WordNetLemmatizer()
    
    #iterate for each tokens
    clean_tokens = []
    for tok in tokens:
        
        if tok not in stopwords.words('english'):
            # lemmatize, normalize case, and remove leading/trailing white space
            clean_tok = lemmatizer.lemmatize(tok).lower().strip()

            clean_tokens.append(clean_tok)
    
    return clean_tokens
    

### 3. Build a machine learning pipeline
- You'll find the [MultiOutputClassifier](http://scikit-learn.org/stable/modules/generated/sklearn.multioutput.MultiOutputClassifier.html) helpful for predicting multiple target variables.

In [37]:
pipeline = Pipeline([
    ('vect', CountVectorizer(tokenizer=tokenize)),
    ('tfid', TfidfTransformer()),
    ('clf', RandomForestClassifier())
])

In [54]:
vec = CountVectorizer(tokenizer=tokenize)
vec.fit_transform(X_train).toarray()

array([[0, 0, 1, 0, 0],
       [1, 1, 0, 0, 0],
       [0, 1, 0, 1, 0],
       [0, 1, 0, 0, 1]], dtype=int64)

In [47]:
X_train

,message,genre
22630,"PADANG, Indonesia, Oct 2, 2009 (AFP) -Quake-tr...",news
21530,The telephone circuit keeps crashing.,news
19403,"On 27 February, in the town of Fianarantsoa, t...",news
20953,"Currently, radioactive iodine is the most comm...",news
17426,The National Meteorological Service (NMS) fore...,news
15868,Kalimantan is again covered by a thick cloud o...,news
25101,The operation was apparently sparked by an att...,news
547,we have transportation. call me -we need help.,direct
21119,"In fact, the solar cookers are much quicker th...",news
14081,These activities include increasing capacity t...,news


### 4. Train pipeline
- Split data into train and test sets
- Train pipeline

In [53]:
X_train, X_test, y_train, y_test = tts(X_onehot,Y,test_size=0.33)

pipeline.fit(X_train.values, y_train)

AttributeError: 'numpy.ndarray' object has no attribute 'lower'

### 5. Test your model
Report the f1 score, precision and recall on both the training set and the test set. You can use sklearn's `classification_report` function here. 

### 6. Improve your model
Use grid search to find better parameters. 

In [ ]:
parameters = 

cv = 

### 7. Test your model
Show the accuracy, precision, and recall of the tuned model.

### 8. Try improving your model further. Here are a few ideas:
* try other machine learning algorithms
* add other features besides the TF-IDF

### 9. Export your model as a pickle file

### 10. Use this notebook to complete `train.py`
Use the template file attached in the Resources folder to write a script that runs the steps above to create a database and export a model based on a new dataset specified by the user.